In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dylanjcastillo/7k-books-with-metadata")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd
books = pd.read_csv(f"{path}/books.csv")

In [ ]:
books

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt


In [ ]:
ax = plt.axes()
sns.heatmap(books.isna().transpose(), cbar=False, ax=ax )

plt.xlabel("Columns")
plt.ylabel("Missing Values")

plt.show()

In [ ]:
import numpy as np
books["missing_description"] = np.where(books["description"].isna(), 1, 0)
books["age_of_book"] = 2024 - books["published_year"]

In [ ]:
columns_of_interest = ["num_pages", "age_of_book", "missing_description", "average_rating"]

correlation_matrix = books[columns_of_interest].corr(method="spearman")

sns.set_theme(style="white")
plt.figure(figsize=(8,6))
heatmap = sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", cbar_kws={"label": "Spearman Correlation"})

heatmap.set_title("COrrealtion heatmap")
plt.show()

In [ ]:
books[(books["description"].isna()) |
    (books["average_rating"].isna()) |
    (books["num_pages"].isna()) |
    (books["published_year"].isna())]

In [ ]:
books_missing = books[~(books["description"].isna()) &
    ~(books["average_rating"].isna()) &
    ~(books["num_pages"].isna()) &
    ~(books["published_year"].isna())]

In [ ]:
books_missing["categories"].value_counts().reset_index().sort_values("count", ascending=False).head(10)

In [ ]:
books_missing["words_in_description"] = books["description"].str.split().str.len()

In [ ]:
books_missing.loc[books_missing["words_in_description"].between(1,4), "description"]

In [ ]:
books_missing_25_words = books_missing[books_missing["words_in_description"] >= 25]

In [ ]:
books_missing_25_words

In [ ]:
books_missing_25_words["titles and subtitles"] = (
    np.where(books_missing_25_words["subtitle"].isna(), books_missing_25_words["title"],
            books_missing_25_words[["title", "subtitle"]].astype(str).agg(" - ".join, axis=1)
    )
)

In [ ]:
books_missing_25_words

In [ ]:
books_missing_25_words["tagged_description"] = books_missing_25_words[["isbn13", "description"]].astype(str).agg(" - ".join, axis=1)

In [ ]:
books_missing_25_words

In [ ]:
(
    books_missing_25_words
    .drop(["subtitle", "missing_description", "words_in_description", "age_of_book"], axis=1)
    .to_csv(f"{path}/books_cleaned.csv", index=False)
)